In [145]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline

file_opioids = 'opioids.csv'
file_overdoses = 'overdoses.csv'
file_prescribers = 'prescriber-info.csv'

In [146]:
opioids = pd.read_csv(file_opioids)
overdoses = pd.read_csv(file_overdoses)
prescribers = pd.read_csv(file_prescribers)

In [147]:
overdoses = overdoses.set_index('State')

In [148]:
overdoses['Population']=overdoses['Population'].str.replace(',', '')
overdoses['Deaths']=overdoses['Deaths'].str.replace(',','')

In [149]:
overdoses[['Population', 'Deaths']]=overdoses[['Population','Deaths']].apply(pd.to_numeric)

In [150]:
max_deaths = overdoses['Deaths'].max()

In [151]:
# per capita measure might be better 
overdoses['Deaths Per Capita'] = overdoses['Deaths']/ overdoses['Population']
max_deaths_per_capita=overdoses['Deaths Per Capita'].max()
print(overdoses.loc[overdoses['Deaths Per Capita']==max_deaths_per_capita])
print(overdoses.loc[overdoses['Deaths']==max_deaths])

print(overdoses['Deaths'].describe())
print(overdoses['Deaths Per Capita'].describe())

               Population  Deaths Abbrev  Deaths Per Capita
State                                                      
West Virginia     1854304     627     WV           0.000338
            Population  Deaths Abbrev  Deaths Per Capita
State                                                   
California    38332521    4521     CA           0.000118
count      50.000000
mean      939.180000
std       898.195486
min        43.000000
25%       281.000000
50%       712.000000
75%      1209.750000
max      4521.000000
Name: Deaths, dtype: float64
count    50.000000
mean      0.000159
std       0.000055
min       0.000059
25%       0.000120
50%       0.000149
75%       0.000191
max       0.000338
Name: Deaths Per Capita, dtype: float64


In [152]:
# set index to prescriber identification number
prescribers=prescribers.set_index('NPI')

In [153]:
print(overdoses.tail())
print(prescribers.tail())

               Population  Deaths Abbrev  Deaths Per Capita
State                                                      
Virginia          8260405     980     VA           0.000119
Washington        6971406     979     WA           0.000140
West Virginia     1854304     627     WV           0.000338
Wisconsin         5742713     853     WI           0.000149
Wyoming            582658     109     WY           0.000187
           Gender State Credentials           Specialty  ABILIFY  \
NPI                                                                
1619019353      F    WA        M.D.     Family Practice        0   
1780693671      F    MI          DO   Internal Medicine        0   
1346270956      M    AZ          MD  Emergency Medicine        0   
1023116514      F    IN        M.D.     Family Practice        0   
1518913672      M    NY        M.D.         Dermatology        0   

            ACETAMINOPHEN.CODEINE  ACYCLOVIR  ADVAIR.DISKUS  AGGRENOX  \
NPI                           

In [154]:
# rename column because it's confusing
prescribers= prescribers.rename({'State':'Abbrev'}, axis=1);

In [155]:
# add prescriber information to overdose df 
prescribers_per_state = []
for index, row in overdoses.iterrows(): 
    prescribers_per_state.append(prescribers.loc[prescribers['Abbrev']==row['Abbrev']].shape[0])
overdoses['Prescribers'] = prescribers_per_state

In [157]:
# prescribers per capita
overdoses['Prescribers Per Capita'] = overdoses['Prescribers']/overdoses['Population']

In [158]:
overdoses.tail()

,Population,Deaths,Abbrev,Deaths Per Capita,Prescribers,Prescribers Per Capita
State,,,,,,
Virginia,8260405,980,VA,0.000119,568,0.000069
Washington,6971406,979,WA,0.000140,578,0.000083
West Virginia,1854304,627,WV,0.000338,199,0.000107
Wisconsin,5742713,853,WI,0.000149,498,0.000087
Wyoming,582658,109,WY,0.000187,38,0.000065


In [159]:
print(overdoses.loc[overdoses['Deaths Per Capita']==max_deaths_per_capita])
print(overdoses.loc[overdoses['Deaths']==max_deaths])

               Population  Deaths Abbrev  Deaths Per Capita  Prescribers  \
State                                                                      
West Virginia     1854304     627     WV           0.000338          199   

               Prescribers Per Capita  
State                                  
West Virginia                0.000107  
            Population  Deaths Abbrev  Deaths Per Capita  Prescribers  \
State                                                                   
California    38332521    4521     CA           0.000118         2562   

            Prescribers Per Capita  
State                               
California                0.000067  
